In [1]:
from target_selector.sql_conn import Database_Handler
import numpy as np
import pandas as pd

## Add 1M_target_list to local db

In [3]:
#db = Database_Handler()
#data = pd.read_csv('/Users/tyler/Downloads/1_million_sample_complete.csv')
#data.to_sql('target_list', db.conn, if_exists='fail')

In [19]:
ra = np.linspace(10, 80, 100)
dec = np.linspace(20, 30, 100)

asdf = []

for i in range(100):
    asdf.append([i, 'POINT({}, {}, 4326)'.format(ra[i], dec[i])])

In [20]:
tb = pd.DataFrame(np.array(asdf), columns=['id', 'coord'])

In [21]:
tb.to_sql('point_test', db.conn, index=False, if_exists='replace')

In [ ]:
db.conn.execute()

## Optimize db

In [21]:
conn = Database_Handler()

In [22]:
def box_filter(c_ra, c_dec, beam_rad, table, cols):

        if c_dec - beam_rad <= - np.pi / 2.0:
            ra_min, ra_max = 0.0, 2.0 * np.pi
            dec_min = -np.pi / 2.0
            dec_max = c_dec + beam_rad

        elif c_dec + beam_rad >= np.pi / 2.0:
            ra_min, ra_max = 0.0, 2.0 * np.pi
            dec_min = c_dec - beam_rad
            dec_max = np.pi / 2.0

        else:
            ra_offset = np.arcsin( np.sin(beam_rad) / np.cos(c_dec))
            ra_min  = c_ra - ra_offset
            ra_max  = c_ra + ra_offset
            dec_min = c_dec - beam_rad
            dec_max = c_dec + beam_rad

        bounds = np.rad2deg([ra_min, ra_max, dec_min, dec_max])

        query = """
                SELECT {cols}
                FROM {table}
                WHERE decl BETWEEN {dec_min} AND {dec_max} 
                AND ra BETWEEN {ra_min} AND {ra_max}\
                """.format(cols = ', '.join(cols), table = table,
                           ra_min = bounds[0], ra_max = bounds[1],
                           dec_min = bounds[2], dec_max = bounds[3])

        return query

In [23]:
c_ra = np.deg2rad(20)
c_dec = np.deg2rad(-28)
beam_rad = np.deg2rad(.5)
query = conn._box_filter(c_ra, c_dec, beam_rad, table = 'target_list', cols = ['ra', 'decl'])

In [24]:
query_between = box_filter(c_ra, c_dec, beam_rad, table = 'target_list', cols = ['ra', 'decl'])

In [27]:
%timeit tb = pd.read_sql(query, con = conn.conn)

4.72 ms ± 38.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%timeit tb = pd.read_sql(query_between, con = conn.conn)

6.82 ms ± 54.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
tb = pd.read_sql(query, con = conn.conn)
tb_between = pd.read_sql(query_between, con = conn.conn)

In [20]:
tb_between.shape, tb.shape

((16, 2), (16, 2))

In [9]:
%timeit tb = conn.select_targets(c_ra, c_dec, beam_rad)

1.2 s ± 31.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
tb = conn.select_targets(c_ra, c_dec, beam_rad)

## Spatial Indexing

In [ ]:
conn = Database_Handler(cred_id = 'observations', db = 'observations')
conn.conn.close()

In [ ]:
try:
    conn.conn.execute("DROP TABLE temp_point_test") 
except:
    pass

try:
    conn.conn.execute("DROP TABLE point_test") 
except:
    pass

In [ ]:
conn.conn.execute('ALTER TABLE point_test ADD coords geometry;')
conn.conn.execute("UPDATE point_test SET coords=(SELECT ST_GeomFromText(geomText) " \
                  "FROM temp_point_test WHERE point_test.id=temp_point_test.id)")
conn.conn.execute("DROP TABLE temp_point_test")   

In [101]:
results = conn.conn.execute("SELECT COLUMN_NAME, DATA_TYPE \
                             FROM INFORMATION_SCHEMA.COLUMNS \
                             WHERE TABLE_NAME = 'point_test'")

In [ ]:
for row in results:
    print (row)

In [107]:
results = conn.conn.execute("SELECT ST_Distance(coords, ST_GeomFromText('POINT( 50.2 25 )')) FROM point_test;")

In [108]:
for row in results:
    print (row)

(19.4,)
(25.4,)
(19.4,)
(25.4,)
